# use class to encapsulate scraping functions

2021/01/03

In [ ]:
from STEP0_paths_open_save import open_files, save_files
from STEP1_scraping_cleaning_merge import scrape, app_detail_dicts
from STEP2_pre_processing import pre_processing
from STEP3_NLP_pipeline import nlp_pipeline

# 1. Scraping

In [1]:
%run -i 'STEP1_scraping_cleaning_merge.py'
from datetime import datetime

initial_panels = ['201812', '201907', '201908', '201912', '202001', '202002']
today = datetime.today()
current_panel = today.strftime("%Y%m")
print(current_panel)

for i in initial_panels:
    o = open_files(initial_panel=i)
    DF = o.open_df_to_id_for_scraping()
    d1 = scrape(initial_panel=i,
                current_panel=current_panel
                ).open_initial_panel_data(
                ).get_appids_from_initial_panel_data(
                ).scraping_apps_according_to_id()

scraping...: 100%|██████████| 8042/8042 [1:02:58<00:00,  2.13it/s] 


202104
start scraping apps with initial panel 201812
The number of IDs are: 5977
Finished scraping app details with initial panel 201812
Saved scarped app details with initial panel 201812
start scraping apps with initial panel 201907
The number of IDs are: 13144
Finished scraping app details with initial panel 201907
Saved scarped app details with initial panel 201907
start scraping apps with initial panel 201908
The number of IDs are: 13015
Finished scraping app details with initial panel 201908
Saved scarped app details with initial panel 201908
start scraping apps with initial panel 201912
The number of IDs are: 8468
Finished scraping app details with initial panel 201912
Saved scarped app details with initial panel 201912
start scraping apps with initial panel 202001
The number of IDs are: 8359
Finished scraping app details with initial panel 202001
Saved scarped app details with initial panel 202001
start scraping apps with initial panel 202002
The number of IDs are: 8042
Finishe

# Panels Track

In [3]:
panels_track = {
'201812':['201812', '201912', '202001', '202003', '202004', '202009', '202010',
          '202011', '202012', '202101', '202102', '202103', '202104'],
'201907':['201907', '201908', '201909', '201912', '202001', '202003', '202004',
          '202009', '202010', '202011', '202012', '202101', '202102', '202103',
          '202104'],
'201908':['201908', '201912', '202001', '202003', '202004', '202009', '202010',
          '202011', '202012', '202101', '202102', '202103', '202104'],
'201912':['201912', '202001', '202003', '202004', '202009', '202010', '202011',
          '202012', '202101', '202102', '202103', '202104'],
'202001':['202001', '202003', '202004', '202009', '202010', '202011', '202012',
          '202101', '202102', '202103', '202104'],
'202002':['202002', '202004', '202009', '202010', '202011', '202012', '202101',
          '202102', '202103', '202104']}

test_initial_panel = '201907'
test_all_panels = ['201907', '201908', '201909', '201912', '202001', '202003', '202004',
          '202009', '202010', '202011', '202012', '202101', '202102', '202103',
          '202104']

# 2. Convert

In [2]:
%run -i 'STEP1_scraping_cleaning_merge.py'

for initial_panel, all_panels in panels_track.items():
    d1 = convert(initial_panel=initial_panel,
                 all_panels=all_panels
                 ).open_app_detail_dict(
                 ).merge_panels_into_single_df()

panel data 201812 has shape:
(4278, 346)
panel data 201907 has shape:
(10576, 398)
panel data 201908 has shape:
(10537, 346)
panel data 201912 has shape:
(7058, 320)
panel data 202001 has shape:
(7027, 294)
panel data 202002 has shape:
(6860, 268)


# 3. Convert to Developer Index and Geocode Addresses
This only need to be run once

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()
%run -i 'STEP1_scraping_cleaning_merge.py'
%run -i 'STEP2_pre_processing.py'

# ***************************************************************************************
for initial_panel, all_panels in panels_track.items():
    o = open_files(initial_panel=initial_panel)
    merged_df = o.open_panel_df(name='MERGED')
    new_class_obj = pre_processing(
                     df=merged_df,
                     initial_panel=initial_panel,
                     all_panels=all_panels)
    DF = new_class_obj.lat_and_long_columns(
                    multiindex=False,
                    consecutive=False)
    s = save_files(df=DF, initial_panel=initial_panel)
    s.save_panel_df(name='dev_index_geocoded')
    print('finished converting', initial_panel, 'dataset')
    print()
    print()

# 5. Impute Missing
Since I will need interval data, so I will only impute from consecutive_panels

I have streamlined code and adjusted containsAds, offersIAP for ALL panels and impute
missing only if all their non-missing panels are the same value.
SAVED in 2021_04_08 / breakthrough_imputed_missing folder.

In [10]:
%run -i 'STEP2_pre_processing.py'

original_vars = ['minInstalls',
                        'score',
                        'reviews',
                        'ratings',
                        'price',
                        'released',
                        'updated',
                        'size',
                        'free',
                        'containsAds',
                        'offersIAP',
                        'contentRating',
                        'genreId']

imputed_vars = ['Imputed' + i for i in original_vars]

# updated value has to strictly follow the previous panel's
# ***************************************************************************************
for initial_panel, all_panels in panels_track.items():
    d1 = pre_processing(
                 initial_panel=initial_panel,
                 all_panels=all_panels
        ).open_merged_df(
        ).count_missing(
                    var_list=original_vars,
                    name='missing_count_before_imputation'
        ).impute_list_of_vars(
                    list_of_vars=original_vars
        ).count_missing(
                    var_list=imputed_vars,
                    name='missing_count_after_imputation')

                      count missing
minInstalls_201812    15           
minInstalls_201912    15           
minInstalls_202001    15           
minInstalls_202003    15           
minInstalls_202004    15           
minInstalls_202009    16           
minInstalls_202010    62           
minInstalls_202011    112          
minInstalls_202012    190          
minInstalls_202101    225          
minInstalls_202102    266          
minInstalls_202103    313          
minInstalls_202104    349          
score_201812          12           
score_201912          12           
score_202001          12           
score_202003          12           
score_202004          12           
score_202009          26           
score_202010          72           
score_202011          122          
score_202012          200          
score_202101          235          
score_202102          276          
score_202103          323          
score_202104          359          
reviews_201812        12    

In [11]:
# take a look at imputed and un-imputed data
%run -i 'STEP2_pre_processing.py'
d2 = pre_processing(
        initial_panel=test_initial_panel,
        all_panels=test_all_panels
).compare_original_and_imputed_var(
    var='free')

/home/naixin/Insync/naixin88@sina.cn/OneDrive/__CODING__/PycharmProjects/GOOGLE_PLAY/scripts/STEP2_pre_processing.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# 6. Delete Missing
Ignore appids that changed developer over time, because we have not imputed developers,
and so we are not deleting the appids that changed developer over time.

I will not use size variable for now because size has to many missing, and I have not come up
with a good way to impute them.

Not going to use updated in my regression, not sure how to impute them the best.

In [3]:
%run -i 'STEP2_pre_processing.py'

original_vars = ['minInstalls',
                        'score',
                        'reviews',
                        'ratings',
                        'price',
                        'released',
                        'updated',
                        'size',
                        'free',
                        'containsAds',
                        'offersIAP',
                        'contentRating',
                        'genreId']

imputed_vars = ['Imputed' + i for i in original_vars]
# ***************************************************************************************
for initial_panel, all_panels in panels_track.items():
    d1 = pre_processing(
            initial_panel=initial_panel,
            all_panels=all_panels
        ).open_dataframe(
            name='imputed_missing'
        ).count_missing(
            var_list=imputed_vars,
            name='missing_count_before_deletion'
        ).collect_appids_to_remove_with_at_least_one_missing(
            varlist=imputed_vars
        ).drop_rows(
        ).count_missing(
            var_list=imputed_vars,
            name='missing_count_after_deletion')

                             count missing
ImputedminInstalls_201812    15           
ImputedminInstalls_201912    15           
ImputedminInstalls_202001    15           
ImputedminInstalls_202003    15           
ImputedminInstalls_202004    15           
ImputedminInstalls_202009    0            
ImputedminInstalls_202010    0            
ImputedminInstalls_202011    0            
ImputedminInstalls_202012    0            
ImputedminInstalls_202101    0            
ImputedminInstalls_202102    0            
ImputedminInstalls_202103    0            
ImputedminInstalls_202104    0            
Imputedscore_201812          10           
Imputedscore_201912          10           
Imputedscore_202001          10           
Imputedscore_202003          10           
Imputedscore_202004          10           
Imputedscore_202009          10           
Imputedscore_202010          10           
Imputedscore_202011          10           
Imputedscore_202012          10           
Imputedscor

# 6. NLP Pipeline

In [1]:
%run -i 'STEP3_NLP_pipeline.py'

for initial_panel, all_panels in panels_track.items():
    d1 = nlp_pipeline(
              tcn='description',
              initial_panel=initial_panel,
              all_panels=all_panels
    ).open_imputed_missing_df(
    ).find_appids_to_remove_before_imputing(
    ).impute_text_cols(
    ).prepare_text_col(
    ).divide_into_subsamples(
    ).generate_save_input_text_col(
    ).tf_idf_transformation(
    ).truncate_svd(
        n_comp=1250,
        random_state=40).kmeans_cluster(
            n_clusters_dict={'full': 350,
                            'game': 350,
                            'nongame': 350},
            init='k-means++',
            random_state=40)

# 7. Combine Dataframes

In [5]:
%run -i 'STEP4_Regression.py'

for initial_panel, all_panels in panels_track.items():
    d1 = combine_dataframes(
             initial_panel=initial_panel,
             all_panels=all_panels
    ).open_imputed_and_deleted_missing_df(
    ).open_predicted_labels_dict(
    )

/home/naixin/Insync/naixin88@sina.cn/OneDrive/__CODING__/PycharmProjects/GOOGLE_PLAY/scripts/STEP4_Regression.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
